<a href="https://colab.research.google.com/github/Fahdlabba/Chat-With-Your-Documents/blob/main/RAG-ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q torch
!pip install -q accelerate  bitsandbytes transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.2 MB/s eta 0:00:00


In [2]:
!pip install langchain pypdf faiss-cpu google-search-results sentence-transformers langchainhub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.8/811.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 56.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 7.1 MB/s eta 0:00:00
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32003 sha256=157d071f94a37d501e7027d9050712cef567600974cfff85e7166322bba59e93
  Stored in directory: /root/.cache/pip/wheels/d

In [112]:
from transformers import AutoTokenizer , AutoModelForCausalLM, BitsAndBytesConfig , pipeline
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.schema.runnable import RunnablePassthrough
import torch

In [4]:
model_name="mistralai/Mistral-7B-Instruct-v0.2"
tokenizer=AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [5]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [7]:
llm=pipeline(
    model=model,
    task="text-generation",
    tokenizer=tokenizer,
    temperature=0.3,
    return_full_text=True,
    do_sample=True,
    max_new_tokens=1000,
)

In [8]:
mistral_llm = HuggingFacePipeline(pipeline=llm)

In [113]:
from google.colab import files
document=files.upload()

Saving fastfacts-what-is-climate-change.pdf to fastfacts-what-is-climate-change.pdf


In [114]:
doc = PyPDFLoader("/content/"+list(document.keys())[0])

In [115]:
doc=doc.load()

In [116]:
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
)

In [117]:
text=text_splitter.split_documents(doc)

In [118]:
db = FAISS.from_documents(text,
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

In [119]:
retriever = db.as_retriever()

In [120]:
template="""[INST] You are helpful Question Answering Ai system, tha try to answer user's questions depending on the given context .
In case you don't have the answer just say I don't have any informations.
You need to reply only for question so if the user's text doesn't seems like question type don't give him any informations .
Here the context you need :
{context}
##
Question : {text}
[/INST]"""

prompt=PromptTemplate(
    input_variables=["text"],
    template=template
)
chain=LLMChain(prompt=prompt,llm=mistral_llm)
rag_chain = (
 {"context": retriever, "text": RunnablePassthrough()}
    | chain
)

In [121]:
result=rag_chain.invoke("What is this document for?")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [122]:
print(result['text'])

 Based on the given context, the documents appear to be pages from a PDF file about "fast facts" related to what climate change is. Therefore, the document is likely for providing information on the topic of climate change. Specifically, the pages contain details about the consequences of climate change, the World Bank's role, and the importance of adapting to climate consequences.


In [124]:
result=rag_chain.invoke("Can you give me a summary of this document?")
dash_line='#'.join(''for i in range(100))
print(dash_line)
print(result['text'])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


###################################################################################################
 The document discusses climate change and its impacts on infrastructure and natural ecosystems. It mentions the current and anticipated effects and the relevance to the Paris Agreement. Three categories of actions are outlined: reducing emissions, providing financial support, and adapting to the impacts. The document also cites sources from the World Bank and other organizations.
